In [1]:
import sys
sys.path.append('/home/users/hswanson13/tbanalysis/') #stupid python
import os
from utils import plotting as pu
from utils import analysis as au

import importlib

In [ ]:
importlib.reload(pu)
importlib.reload(au)

from scipy.stats import crystalball
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

def crystal_ballin(arr:np.ndarray, N:float, mean:float, sigma:float, beta:float, m:float) -> np.ndarray:
    """
    N:     amplitude of the distrobution \n
    mean:  the mean of the guassian part of the crystal ball \n
    sigma: the width of the gaussian part of the crystal ball \n
    beta:  the point where the pdf changes from a power law to a gaussian distrobution \n
    m:     the power of the power-law tail \n
    """
    #convert array like this, see definition here
    #https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.crystalball.html
    x = (arr-mean)/sigma
    return N*crystalball.pdf(x,beta,m)


tb_plot = pu.TBplot("/home/users/hswanson13/tbanalysis/output_analysis_data/new_plot_test3/start_4600_stop_4899_bias_270_offset_15.0_energy_5.0_power_i1_file_from_DESY_module_36_LP2_20_cubicLM_unbinned")
tb_plot.hist_bins['dt'] = (300, 5, 18)
tb_plot.hist_bins['dt_corr'] = (30,-0.25,0.25)
pix = (9,8)

dt_hist = tb_plot.get_hist('dt_corr', pix=pix)
#tb_plot.histo1D('dt_corr', pix=pix)

bin_centers, hist_values = dt_hist.axes.centers[0], dt_hist.values()

hist_uncert = np.sqrt(dt_hist.variances())
#https://github.com/scikit-hep/hist/blob/6fb3ecd07d1f9a4758cd5d5ccf89559ed572ca9a/src/hist/plot.py#L150
mask = hist_uncert != 0.0

N = 200
mean = (hist_values * bin_centers).sum() / hist_values.sum()
sigma = (hist_values * np.square(bin_centers - mean)).sum() / hist_values.sum()
beta = abs(mean-0.08)
m = 2

popt, pcov = curve_fit(
    crystal_ballin, 
    bin_centers[mask], 
    hist_values[mask], 
    # sigma=hist_uncert[mask],
    # absolute_sigma=True,
    p0=[N, mean, sigma, beta, m]
)

print(popt[2]*1000)
fig, ax = plt.subplots()
dt_hist.plot1d(ax=ax)

x = np.linspace(-5,5,1000)
ax.plot(
    bin_centers,
    crystal_ballin(bin_centers, *popt)
    # x,
    # crystal_ballin(x, 10, 0, 1,1,2)
)

plt.axvline(popt[1]-popt[2]*popt[3])


In [ ]:
importlib.reload(pu)
importlib.reload(au)

from scipy.stats import crystalball
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from scipy.integrate import quad


def double_crystal_ball(x, N, mean, sigma, beta_L, m_L, beta_R, m_R):
    """
    N:     amplitude of the distrobution \n
    mean:  the mean of the guassian part of the crystal ball \n
    sigma: the width of the gaussian part of the crystal ball \n
    beta_L or beta_R:  the point where the pdf changes from a power law to a gaussian distrobution (number of standard deviations away from the mean) \n
    m_L or n_R:     the power of the power-law tail \n
    """
    x = (x-mean)/sigma

    # if beta_L == 0 and beta_R == 0:
    #     return np.exp(-x[mid_sel]**2 / 2)
    
    #convert array like this, see definition here
    #https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.crystalball.html
    A1 = (m_L / abs(beta_L))**m_L * np.exp(-beta_L**2 / 2)
    B1 = m_L / abs(beta_L) - abs(beta_L)
    #N1 = 1.0 / (quad(lambda t: np.exp(-t**2 / 2), -np.inf, beta_L)[0] + A1 / (m_L - 1.0))
    
    A2 = (m_R / abs(beta_R))**m_R * np.exp(-beta_R**2 / 2)
    B2 = m_R / abs(beta_R) - abs(beta_R)
    #N2 = 1.0 / (quad(lambda t: np.exp(-t**2 / 2), -beta_R, np.inf)[0] + A2 / (m_R - 1.0))
    
    left_sel = x <= -beta_L
    left_power = A1*(B1 - x[left_sel])**-m_L

    right_sel = x >= beta_R
    right_power = A2*(B2 + x[right_sel])**-m_R

    mid_sel = ((x<beta_R) & (x>-beta_L))
    mid_gauss = np.exp(-x[mid_sel]**2 / 2)

    #print(len(left_power), len(mid_gauss), len(right_power))
    return N*np.concatenate([left_power, mid_gauss, right_power], axis=0)

tb_plot = pu.TBplot("/home/users/hswanson13/tbanalysis/output_analysis_data/new_plot_test3/start_4600_stop_4899_bias_270_offset_15.0_energy_5.0_power_i1_file_from_DESY_module_36_LP2_20_cubicLM_unbinned")
tb_plot.hist_bins['dt'] = (150, 11, 13)
tb_plot.hist_bins['dt_corr'] = (30,-0.25,0.25)
pix = (9,8)


dt_hist = tb_plot.get_hist('dt_corr', pix=pix)
#tb_plot.histo1D('dt_corr', pix=pix)
bin_centers, hist_values = dt_hist.axes.centers[0], dt_hist.values()

N = np.max(hist_values)
mean = (hist_values * bin_centers).sum() / hist_values.sum()
sigma = (hist_values * np.square(bin_centers - mean)).sum() / hist_values.sum()
beta_L = abs(mean-0.2)
m_L = 8
beta_R = abs(mean+0.2)
m_R = 8

hist_uncert = np.sqrt(dt_hist.variances())
#https://github.com/scikit-hep/hist/blob/6fb3ecd07d1f9a4758cd5d5ccf89559ed572ca9a/src/hist/plot.py#L150
mask = hist_uncert != 0.0
popt, pcov = curve_fit(
    double_crystal_ball, 
    bin_centers, 
    hist_values, 
    sigma=hist_uncert[mask],
    absolute_sigma=True,
    p0= [N, mean, sigma, beta_L, m_L, beta_R, m_R],
    # p0= [N, mean, sigma, beta, m],
    # bounds=(
    #     # (0, -25, 0.0001,  0.020*4, 1.1, 0.020*4,   1.1),
    #     (0, 0, 0.0001,  sigma, 1.1, sigma,   1.1),
    #     (np.inf, 25,  np.inf,  np.inf,  np.inf, np.inf, np.inf)
    # )
)
r = hist_values - double_crystal_ball(bin_centers, *popt)
chisq = np.sum((r[mask]/hist_uncert[mask])**2)
deg_freedom = sum(hist_values) - 7
red_chisq = chisq/deg_freedom
fig, ax = plt.subplots()
dt_hist.plot1d(ax=ax)
ax.plot(
    bin_centers,
    double_crystal_ball(bin_centers, *popt),
    label=f"Resolution={popt[2]*1000:.2f} ps \nReduced Chi2={red_chisq:.2f}"
)
plt.legend()
plt.axvline(popt[1]-popt[2]*popt[3])
plt.axvline(popt[1]+popt[2]*popt[5])

# z = np.linspace(0,10,11)

# x = np.linspace(-5,5,1000)
# fig, ax = plt.subplots()

# ax.plot(
#     x,
#     double_crystal_ball(x, 1,0,1, 1,6, 3,2)
# )

tb_plot.histo1D('dt_corr',pix=pix)